V1 требует больше памяти чем на 3080 ti. В последний раз пытался выделить 13 гб VRAM
Поэтому переходим на V2 в который выполнена небольшая оптимизация для экономии VRAM

In [3]:
# === Шаг 1: Импорт всех библиотек ===
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os

# === Шаг 2: Настройка окружения и загрузка моделей ===

# --- Настройка устройства ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

# --- Загрузка Segment Anything Model (SAM) ---
print("Загрузка модели SAM...")
sam_checkpoint = "/app/MODELS/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)
print("SAM готов.")

# --- Загрузка CLIP ---
print("Загрузка модели CLIP...")
# 'openai/clip-vit-large-patch14' - одна из самых мощных моделей CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
print("CLIP готов.")

Используемое устройство: cuda
Загрузка модели SAM...
SAM готов.
Загрузка модели CLIP...


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


CLIP готов.


In [5]:
# === Шаг 3: Сегментация и Классификация ===

# --- Настройки для текущего запуска ---
image_path = "/app/CODE/Vistino20241014_E57/1_normals.jpg"
image_name = os.path.basename(image_path)
semantic_labels = [
    "человек", "металлическая конструкция", "автомобиль", "здание",
    "деревья", "растения", "земля", "небо", "другой объект"
]
min_mask_area = 1000 # Минимальная площадь маски в пикселях

# --- Загрузка и обработка изображения ---
print(f"\n--- Обработка изображения: {image_name} ---")
image_bgr = cv2.imread(image_path)
if image_bgr is None:
    print(f"Ошибка: не удалось загрузить изображение по пути {image_path}")
    # Убедимся, что переменная masks не существует, чтобы не вызывать ошибку дальше
    if 'masks' in locals():
        del masks
else:
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    print("Изображение загружено, запускаю сегментацию...")
    masks = mask_generator.generate(image_rgb)
    print(f"Сегментация завершена. Найдено {len(masks)} масок.")

    # Фильтрация масок по площади
    original_mask_count = len(masks)
    masks = [m for m in masks if m['area'] > min_mask_area]
    print(f"Фильтрация по площади: было {original_mask_count}, стало {len(masks)}.")

    # Классификация отфильтрованных масок
    semantic_results = []
    print("\nЗапускаю классификацию масок с помощью CLIP...")
    for i, mask_data in enumerate(masks):
        x, y, w, h = mask_data['bbox']
        pil_image = Image.fromarray(image_rgb[y:y+h, x:x+w])
        inputs = clip_processor(text=semantic_labels, images=pil_image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = clip_model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1)
        mask_data['label'] = semantic_labels[probs.argmax()]
        mask_data['confidence'] = probs.max().item()
        semantic_results.append(mask_data)
        print(f"  Маска {i+1}/{len(masks)}: {mask_data['label']} (уверенность: {mask_data['confidence']:.2f})")
    print("Классификация завершена.")

# === Шаг 4: Визуализация с семантикой (улучшенная и более надежная функция) ===

def visualize_semantic_masks(image_rgb, results):
    """Более надежная функция для визуализации."""
    # Создаем копию изображения для рисования
    vis_image = image_rgb.copy()
    
    # Сортируем маски, чтобы сначала рисовались самые большие
    sorted_results = sorted(results, key=(lambda x: x['area']), reverse=True)
    
    for res in sorted_results:
        mask = res['segmentation']
        
        # Создаем цветную маску
        color = np.random.randint(0, 255, 3) # Цвет от 0 до 255
        
        # Накладываем маску на изображение с прозрачностью
        vis_image[mask] = cv2.addWeighted(vis_image[mask], 0.5, np.full_like(vis_image[mask], color), 0.5, 0)
    
    # Рисуем изображение с масками один раз
    plt.figure(figsize=(20, 15))
    plt.imshow(vis_image)
    
    # Добавляем текстовые метки поверх
    for res in sorted_results:
        label = res['label']
        confidence = res['confidence']
        
        if confidence > 0.6 and label != "другой объект":
            x, y, w, h = res['bbox']
            center_x, center_y = x + w // 2, y + h // 2
            plt.text(center_x, center_y, f"{label}\n{confidence:.2f}", 
                     color='white', backgroundcolor='black',
                     fontsize=9, ha='center', va='center')
            
    plt.axis('off')
    plt.title(f"Семантическая сегментация для '{os.path.basename(image_path)}'")
    plt.show()

# Запускаем визуализацию, только если 'masks' существует и не пуста
if 'masks' in locals() and masks:
    visualize_semantic_masks(image_rgb, semantic_results)


--- Обработка изображения: 1_normals.jpg ---
Изображение загружено, запускаю сегментацию...


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.45 GiB (GPU 0; 12.00 GiB total capacity; 13.32 GiB already allocated; 0 bytes free; 18.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF